<a href="https://colab.research.google.com/github/DreRnc/ExplainingExplanations/blob/torch/Explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset : **E-SNLI**. \
Model : **Small T5**.

In [1]:
colab = False

In [2]:
if colab:
    !git clone https://github.com/DreRnc/ExplainingExplanations.git
    %cd ExplainingExplanations
    %pip install -r requirements.txt

# 1.0 Preparation


## 1.1 Loading Dataset

In [3]:
from datasets import load_dataset

dataset = load_dataset("esnli")

/Users/dre/Desktop/GitRepo/DreRnc/ExplainingExplanations/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
training_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

print("Shape of training_set: ", training_set.shape)
print("Shae of validation_set: ", validation_set.shape)
print("Shape of test_set: ", test_set.shape)

Shape of training_set:  (549367, 6)
Shae of validation_set:  (9842, 6)
Shape of test_set:  (9824, 6)


In [5]:
training_set[0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

In [6]:
n_train = n_valid = n_test = 5000

train_small = training_set.select(range(n_train))
valid_small = validation_set.select(range(n_valid))
test_small = test_set.select(range(n_test))

print("Shape of train_small: ", train_small.shape)
print("Shape of valid_small: ", valid_small.shape)
print("Shape of test_small: ", test_small.shape)

Shape of train_small:  (5000, 6)
Shape of valid_small:  (5000, 6)
Shape of test_small:  (5000, 6)


## 1.2 Loading T5 Model

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Test **zero-shot** on a random task.

In [8]:
input_ids = tokenizer(
    "translate English to French: Hello Dre, I think the English version is ok for us.",
    return_tensors="pt",
).input_ids
outputs = model.generate(input_ids, max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=100))

Bonjour Dre, je pense que la version anglaise est bonne pour nous.


## 1.3 Zero-shot example to Verify Everything is Working

In [9]:
from src.utils import generate_prompt_mnli

In [10]:
example = training_set[0]
example

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

Generating the prompt:

<b><u> mnli hypothesis: </b></u> The St. Louis Cardinals have always won. <b><u> premise: </b></u> yeah well losing is i mean i’m i’m originally from Saint Louis and Saint Louis Cardinals when they were there were uh a mostly a losing team but

Output:
* 0: Entailment
* 1: Neutral
* 2: Contradiction

In [11]:
prompt = generate_prompt_mnli(example)
prompt

'mnli hypothesis: A person is training his horse for a competition. premise: A person on a horse jumps over a broken down airplane.'

In [12]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model.generate(input_ids)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

neutral


/Users/dre/Desktop/GitRepo/DreRnc/ExplainingExplanations/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## 1.4  Tokenize the dataset

In [13]:
train_small.info

DatasetInfo(description='', citation='', homepage='', license='', features={'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None), 'explanation_1': Value(dtype='string', id=None), 'explanation_2': Value(dtype='string', id=None), 'explanation_3': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='esnli', config_name='plain_text', version=0.0.2, splits={'train': SplitInfo(name='train', num_bytes=107611996, num_examples=549367, shard_lengths=None, dataset_name='esnli'), 'validation': SplitInfo(name='validation', num_bytes=3416319, num_examples=9842, shard_lengths=None, dataset_name='esnli'), 'test': SplitInfo(name='test', num_bytes=3379781, num_examples=9824, shard_lengths=None, dataset_name='esnli')}, download_checksums={'hf://datasets/esnli@f01d1430c0ad65a3785d06079a0e01e015e769b8/plain_text/

In [14]:
train_small[0]["prompt"] = "hello"

train_small[0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

In [15]:
train_small.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'explanation_1': Value(dtype='string', id=None),
 'explanation_2': Value(dtype='string', id=None),
 'explanation_3': Value(dtype='string', id=None)}

In [16]:
from functools import partial
from src.utils import tokenize_function
import time

In [17]:
tokenize_mapping = partial(tokenize_function, tokenizer=tokenizer)

In [31]:
time_init = time.time()
train_small_tokenized = train_small.map(tokenize_mapping, batched=True).with_format("torch", columns=["input_ids", "attention_mask"])
valid_small_tokenized = valid_small.map(tokenize_mapping, batched=True).with_format("torch", columns=["input_ids", "attention_mask"])
test_small_tokenized = test_small.map(tokenize_mapping, batched=True).with_format("torch", columns=["input_ids", "attention_mask"])
time_end = time.time()

print("Time taken to tokenize: ", time_end - time_init)
print("Shape of train_small_tokenized: ", train_small_tokenized.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized.shape)

Time taken to tokenize:  0.05743098258972168
Shape of train_small_tokenized:  (5000, 8)
Shape of valid_small_tokenized:  (5000, 8)
Shape of test_small_tokenized:  (5000, 8)


In [32]:
train_small_tokenized.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'explanation_1': Value(dtype='string', id=None),
 'explanation_2': Value(dtype='string', id=None),
 'explanation_3': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [20]:
train_small_tokenized = train_small_tokenized.rename_column("label", "labels")
train_small_tokenized = train_small_tokenized.remove_columns(["premise", "hypothesis", "explanation_1", "explanation_2", "explanation_3"])

valid_small_tokenized = valid_small_tokenized.rename_column("label", "labels")
valid_small_tokenized = valid_small_tokenized.remove_columns(["premise", "hypothesis", "explanation_1", "explanation_2", "explanation_3"])

test_small_tokenized = test_small_tokenized.rename_column("label", "labels")
test_small_tokenized = test_small_tokenized.remove_columns(["premise", "hypothesis", "explanation_1", "explanation_2", "explanation_3"])

In [21]:
train_small_tokenized.features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [22]:
from torch.utils.data import DataLoader

In [23]:
train_dataloader = DataLoader(train_small_tokenized, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(valid_small_tokenized, batch_size=8)
test_dataloader = DataLoader(test_small_tokenized, batch_size=8)

# 2.0 Task 1: Zero-shot evaluation

In [24]:
from src.utils import convert_labels
from tqdm import tqdm
import torch
import evaluate

In [25]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model.to(device)
device

device(type='cpu')

In [26]:
metric = evaluate.load("accuracy")
model.eval()

progress_bar = tqdm(range(len(test_dataloader)))

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model.generate(**batch)
        output = tokenizer.batch_decode(outputs, skip_special_tokens=True, max_length=100)
        preds = convert_labels(output)

    metric.add_batch(
        predictions=preds,
        references=batch["labels"],
    )

    progress_bar.update(1)

metric.compute()

  0%|          | 0/625 [00:00<?, ?it/s]/Users/dre/Desktop/GitRepo/DreRnc/ExplainingExplanations/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyError: 'labels'

# 3.0 Task 2: Fine tuning without explanations

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
import torch.nn as nn
loss = nn.CrossEntropyLoss()

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:

        optimizer.zero_grad()

        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model.generate(**batch)
        output = tokenizer.batch_decode(outputs, skip_special_tokens=True, max_length=100)
        preds = convert_labels(output)

        loss_value = loss(preds, batch["labels"])
        loss_value.backward()

        optimizer.step()
        lr_scheduler.step()

        progress_bar.update(1)

  0%|          | 0/1875 [02:48<?, ?it/s]


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not list

# 4.0 Task 4: Making the model generate explanations